<a href="https://colab.research.google.com/github/i1idan/schizophrenia-diagnosis-eeg-signals/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Clone the Repository

In [ ]:
!git clone https://github.com/i1idan/schizophrenia-diagnosis-eeg-signals.git

In [ ]:
import os
os.chdir('/content/schizophrenia-diagnosis-eeg-signals')
!git pull origin main

# Install Requirements

In [ ]:
!pip install -r requirements.txt

# Get Data

In [ ]:
!gdown --id 1jnWHWrArzJQIvny0cQkfPP42hEJAp_56
!mv DATA.mat /content/schizophrenia-diagnosis-eeg-signals/data/DATA.mat

# Training Models

In [2]:
# Number of training in a sequence
multi_train = 10
epochs = 200
model_name = "WaveletCustom"
data_path = "./data/DATA.mat"
# checkpoints = "/content/drive/MyDrive/schizophrenia/checkpoints"
checkpoints = "./checkpoints"
batch_size = 4
early_stopping = 100
reduce_lr = 50
seed = 1234

In [ ]:
csv_files = []
for n in range(multi_train):
    dir_name = f"{n}"
    new_seed = seed + n
    # preserve reproducibility
    !PYTHONHASHSEED=0
    !TF_DETERMINISTIC_OPS=0
    !TF_CUDNN_DETERMINISTIC=0
    !python train.py --model-name $model_name --epochs $epochs --seed $new_seed --dir-name $dir_name --checkpoints $checkpoints --batch-size $batch_size
    print(f"-----------------------------train {n} is done! ----------------------------------")

[INFO] Model:WaveletCustom is loaded ...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 110, 110, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 108, 108, 8)       80        
                                                                 
 conv2d_1 (Conv2D)           (None, 107, 107, 8)       264       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 53, 53, 8)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 53, 53, 8)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 16)        528       
               

213/213 [==============================] - 2s 11ms/step - loss: 0.0384 - accuracy: 0.9859 - val_loss: 0.1452 - val_accuracy: 0.9484 - lr: 0.0010
Epoch 30/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0406 - accuracy: 0.9812 - val_loss: 0.1856 - val_accuracy: 0.9531 - lr: 0.0010
Epoch 31/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0667 - accuracy: 0.9765 - val_loss: 0.1804 - val_accuracy: 0.9484 - lr: 0.0010
Epoch 32/200
213/213 [==============================] - 2s 10ms/step - loss: 0.0470 - accuracy: 0.9812 - val_loss: 0.1445 - val_accuracy: 0.9343 - lr: 0.0010
Epoch 33/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0607 - accuracy: 0.9788 - val_loss: 0.1495 - val_accuracy: 0.9437 - lr: 0.0010
Epoch 34/200
213/213 [==============================] - 2s 11ms/step - loss: 0.0615 - accuracy: 0.9800 - val_loss: 0.1081 - val_accuracy: 0.9577 - lr: 0.0010
Epoch 35/200
213/213 [==============================] - 1s 6ms/step 

213/213 [==============================] - 1s 5ms/step - loss: 0.0236 - accuracy: 0.9929 - val_loss: 0.0923 - val_accuracy: 0.9671 - lr: 0.0010
Epoch 82/200
213/213 [==============================] - 1s 6ms/step - loss: 0.0086 - accuracy: 0.9965 - val_loss: 0.0844 - val_accuracy: 0.9906 - lr: 0.0010
Epoch 83/200
213/213 [==============================] - 1s 6ms/step - loss: 0.0412 - accuracy: 0.9871 - val_loss: 0.0727 - val_accuracy: 0.9859 - lr: 0.0010
Epoch 84/200
213/213 [==============================] - 2s 8ms/step - loss: 0.0521 - accuracy: 0.9906 - val_loss: 0.0785 - val_accuracy: 0.9812 - lr: 0.0010
Epoch 85/200
213/213 [==============================] - 1s 6ms/step - loss: 0.0113 - accuracy: 0.9976 - val_loss: 0.0933 - val_accuracy: 0.9718 - lr: 0.0010
Epoch 86/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0072 - accuracy: 0.9953 - val_loss: 0.0820 - val_accuracy: 0.9765 - lr: 0.0010
Epoch 87/200
213/213 [==============================] - 1s 5ms/step - l

213/213 [==============================] - 1s 5ms/step - loss: 0.0141 - accuracy: 0.9941 - val_loss: 0.0943 - val_accuracy: 0.9812 - lr: 0.0010
Epoch 134/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0117 - accuracy: 0.9965 - val_loss: 0.0863 - val_accuracy: 0.9859 - lr: 0.0010
Epoch 135/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0072 - accuracy: 0.9965 - val_loss: 0.0734 - val_accuracy: 0.9812 - lr: 0.0010
Epoch 136/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0249 - accuracy: 0.9918 - val_loss: 0.0294 - val_accuracy: 0.9812 - lr: 0.0010
Epoch 137/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0046 - accuracy: 0.9976 - val_loss: 0.0489 - val_accuracy: 0.9812 - lr: 0.0010
Epoch 138/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0363 - accuracy: 0.9929 - val_loss: 0.0984 - val_accuracy: 0.9765 - lr: 0.0010
Epoch 139/200
213/213 [==============================] - 1s 5ms/st

213/213 [==============================] - 1s 6ms/step - loss: 8.2491e-05 - accuracy: 1.0000 - val_loss: 0.0812 - val_accuracy: 0.9765 - lr: 5.0000e-04
Epoch 183/200
213/213 [==============================] - 1s 6ms/step - loss: 0.0012 - accuracy: 0.9988 - val_loss: 0.0221 - val_accuracy: 0.9859 - lr: 5.0000e-04
Epoch 184/200
213/213 [==============================] - 1s 5ms/step - loss: 1.5037e-05 - accuracy: 1.0000 - val_loss: 0.0211 - val_accuracy: 0.9906 - lr: 5.0000e-04
Epoch 185/200
213/213 [==============================] - 1s 5ms/step - loss: 1.5591e-05 - accuracy: 1.0000 - val_loss: 0.0213 - val_accuracy: 0.9906 - lr: 5.0000e-04
Epoch 186/200
213/213 [==============================] - 1s 5ms/step - loss: 4.2639e-04 - accuracy: 1.0000 - val_loss: 0.0161 - val_accuracy: 0.9906 - lr: 5.0000e-04
Epoch 187/200
213/213 [==============================] - 1s 6ms/step - loss: 1.5118e-04 - accuracy: 1.0000 - val_loss: 0.0168 - val_accuracy: 0.9906 - lr: 5.0000e-04
Epoch 188/200
213/213 

...

213/213 [==============================] - 1s 7ms/step - loss: 9.2411e-04 - accuracy: 1.0000 - val_loss: 0.3618 - val_accuracy: 0.9390 - lr: 5.0000e-04
Epoch 165/200
213/213 [==============================] - 1s 7ms/step - loss: 0.0047 - accuracy: 0.9976 - val_loss: 0.2265 - val_accuracy: 0.9484 - lr: 5.0000e-04
Epoch 166/200
213/213 [==============================] - 1s 7ms/step - loss: 0.0054 - accuracy: 0.9988 - val_loss: 0.2516 - val_accuracy: 0.9437 - lr: 5.0000e-04
Epoch 167/200
213/213 [==============================] - 1s 7ms/step - loss: 0.0179 - accuracy: 0.9941 - val_loss: 0.2143 - val_accuracy: 0.9531 - lr: 5.0000e-04
Epoch 168/200
213/213 [==============================] - 1s 7ms/step - loss: 0.0039 - accuracy: 0.9976 - val_loss: 0.2878 - val_accuracy: 0.9531 - lr: 5.0000e-04
Epoch 169/200
213/213 [==============================] - 1s 7ms/step - loss: 1.0905e-04 - accuracy: 1.0000 - val_loss: 0.2934 - val_accuracy: 0.9484 - lr: 5.0000e-04
Epoch 170/200
213/213 [===========

[INFO] H.shape: (14,), each sample has (channels, values): (19, 180000) !
[INFO] S.shape: (14,), each sample has (channels, values): (19, 180000) !
[INFO] S.shape: (14,)
[INFO] CHAN.shape: (1, 19)
[INFO] Fs.shape: (1, 1)
[INFO] healthy_samples: 14, schizo_samples: 14, channel_size: 19, and sub: 15000
[INFO] normal samples: (532, 112, 112)
[INFO] schizo samples: (532, 112, 112)
[INFO] labels: (1064,)
[INFO] data: (1064, 112, 112, 1)
[INFO] x_train.shape: (851, 112, 112, 1), y_train.shape: (851,)
[INFO] x_test.shape: (213, 112, 112, 1), y_test.shape: (213,)
[INFO] Started the training for model: WaveletCustom ...
[INFO] Saving params!
[INFO] Params are successfully saved!
[INFO] Training with the following arguments Namespace(seed=1236, model_name='WaveletCustom', data_path='./data/DATA.mat', epochs=200, batch_size=4, checkpoints='./checkpoints', early_stopping=100, reduce_lr=50, dir_name='2')
Epoch 1/200
213/213 [==============================] - 5s 14ms/step - loss: 0.6561 - accuracy: 

213/213 [==============================] - 1s 6ms/step - loss: 0.1020 - accuracy: 0.9718 - val_loss: 0.3758 - val_accuracy: 0.9014 - lr: 0.0010
Epoch 48/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0348 - accuracy: 0.9894 - val_loss: 0.5666 - val_accuracy: 0.8873 - lr: 0.0010
Epoch 49/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0649 - accuracy: 0.9847 - val_loss: 0.4311 - val_accuracy: 0.9061 - lr: 0.0010
Epoch 50/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0301 - accuracy: 0.9894 - val_loss: 0.3496 - val_accuracy: 0.9202 - lr: 0.0010
Epoch 51/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0189 - accuracy: 0.9906 - val_loss: 0.4061 - val_accuracy: 0.9249 - lr: 0.0010
Epoch 52/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0322 - accuracy: 0.9882 - val_loss: 0.3746 - val_accuracy: 0.9108 - lr: 0.0010
Epoch 53/200
213/213 [==============================] - 1s 5ms/step - l

213/213 [==============================] - 1s 5ms/step - loss: 0.0062 - accuracy: 0.9988 - val_loss: 0.6937 - val_accuracy: 0.9249 - lr: 5.0000e-04
Epoch 99/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.6881 - val_accuracy: 0.9249 - lr: 5.0000e-04
Epoch 100/200
213/213 [==============================] - 1s 5ms/step - loss: 7.1876e-04 - accuracy: 1.0000 - val_loss: 0.6818 - val_accuracy: 0.9296 - lr: 5.0000e-04
Epoch 101/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0147 - accuracy: 0.9965 - val_loss: 0.4133 - val_accuracy: 0.9202 - lr: 5.0000e-04
Epoch 102/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0202 - accuracy: 0.9965 - val_loss: 0.3103 - val_accuracy: 0.9437 - lr: 5.0000e-04
Epoch 103/200
213/213 [==============================] - 1s 5ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.3180 - val_accuracy: 0.9484 - lr: 5.0000e-04
Epoch 104/200
213/213 [================

[INFO] H.shape: (14,), each sample has (channels, values): (19, 180000) !
[INFO] S.shape: (14,), each sample has (channels, values): (19, 180000) !
[INFO] S.shape: (14,)
[INFO] CHAN.shape: (1, 19)
[INFO] Fs.shape: (1, 1)
[INFO] healthy_samples: 14, schizo_samples: 14, channel_size: 19, and sub: 15000
[INFO] normal samples: (532, 112, 112)
[INFO] schizo samples: (532, 112, 112)
[INFO] labels: (1064,)
[INFO] data: (1064, 112, 112, 1)
[INFO] x_train.shape: (851, 112, 112, 1), y_train.shape: (851,)
[INFO] x_test.shape: (213, 112, 112, 1), y_test.shape: (213,)
[INFO] Started the training for model: WaveletCustom ...
[INFO] Saving params!
[INFO] Params are successfully saved!
[INFO] Training with the following arguments Namespace(seed=1237, model_name='WaveletCustom', data_path='./data/DATA.mat', epochs=200, batch_size=4, checkpoints='./checkpoints', early_stopping=100, reduce_lr=50, dir_name='3')
Epoch 1/200
213/213 [==============================] - 5s 14ms/step - loss: 0.6482 - accuracy: 

# Get Metrics

In [ ]:
from utils.group_metrics import get_mean_std
import os

csv_files = [os.path.join(checkpoints, model_name ,f"{n}", "log.csv") for n in range(multi_train)]
metrics = get_mean_std(csv_files, 
                       arguments=("accuracy", "loss", "val_accuracy", "val_loss"),
                       operators=(max, min, max, min)
                      )
print(metrics)